# Selenium Scraping

In [35]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
chrome_path = r'/chromedriver/chromedriver.exe'

In [3]:
#Chrome setting. Just ignore this
options = webdriver.ChromeOptions()
prefs = {'profile.default_content_settings.popups': 0, 'download.default_directory': '/Users/hongfan/user-generated-content-analytics'}
options.add_experimental_option('prefs', prefs)

In [4]:
driver = webdriver.Chrome(chrome_path, chrome_options=options) 

In [5]:
url="https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/"

In [ ]:
num_pages = 2575

In [6]:
# convert model names to brands
import csv

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
file = open('models.csv', 'r')
csv_file = csv.reader(file)

# don't store brand names that are weird from the csv file. like 'problems' and 'car' those arent brand names
weird_brands = ['car', 'hyndai kia', 'hyundai,', 'hyundai.', 'kia,', 'kia.', 
                'mercedes benz', 'mercedes-benz', 'nissan.', 'problem', 'seat', 'toyata','volkwagen' ]

# store the brand names
brands = []
distinct_brands = []
for i in csv_file:
    if(i[0] in weird_brands):
        continue
    else:
        brands.append(i)
        if(i[0] not in distinct_brands):
            distinct_brands.append(i[0])
#     parsedData = parsedData.replace(i[1], i[0])

print('Finished storing this csv file into a list')
# close the file
file.close()

Finished storing this csv file into a list


In [69]:
output_csv = []

for page_number in range(1, 100):
    if(page_number == 1):
        new_url = url
    else:
        new_url = url + "p" + str(page_number)
    print(new_url)
    
    # Take a website that you want to scrape
    driver.get(new_url)  # It will send you to that website
    
    # now find each post
    posts = driver.find_elements_by_class_name('Comment')
    
    for i in posts:
        # username 
        username = i.find_element_by_class_name('Username').text
    
        # date
        date = i.find_element_by_tag_name('time').text
    
        # extract the comment without blockquotes!
        body_wrap = i.find_element_by_class_name('Item-BodyWrap')
        item_body = body_wrap.find_element_by_class_name('Item-Body')
        comment = item_body.find_element_by_class_name('userContent')
        text = comment.text # raw text with blockquotes maybe

        quotes = comment.find_elements_by_tag_name('blockquote') # extracts each blockquote into a list 

        for j in quotes:
            text = text.replace(j.text, "")
        
        # Let's do some string manipulation
        text = text.replace('\n', '')
        text = text.lower()

        # deal with brand name
        for k in brands:
            text = text.replace(k[1], k[0])
    
        # create a tuple with Date, Userid, message and store into output_csv
        row = date, username, text
        output_csv.append(row)
        
    print('Page ' + str(page_number) + ' complete')


https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/
Page 1 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p2
Page 2 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p3
Page 3 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p4
Page 4 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p5
Page 5 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p6
Page 6 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p7
Page 7 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p8
Page 8 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p9
Page 9 complete
https://forums.edmunds.com/dis

Page 75 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p76
Page 76 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p77
Page 77 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p78
Page 78 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p79
Page 79 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p80
Page 80 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p81
Page 81 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p82
Page 82 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p83
Page 83 complete
https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p84
Page 84 co

In [73]:
output_csv[65]

('June 2014',
 'stever',
 '@abacomike, missed your reply earlier - we hit the art museum today and enjoyed a "small paintings" exhibit from cheech marin\'s collection. we\'re heading back to the up tomorrow, having enjoyed a nice spell of "mild" weather here this week - don\'t think it got over 95.the ac on the \'09 grand caravan we picked up a few weeks back is blowing nice and cold at 125k.')

## Throw output_csv list into a csv file 

In [74]:
# output into csv file
# throw this into a text file just in case
with open('seleniumfinalcomments.csv','w', encoding = 'utf-8', newline = '') as out:
    csv_out = csv.writer(out)
    csv_out.writerow (['date','username', 'comment'])
    for row in output_csv:
        csv_out.writerow(row)

print('write csv is complete')

write csv is complete


In [75]:
# sanity check
import pandas as pd
pd.read_csv('seleniumfinalcomments.csv')

,date,username,comment
0,June 2014,graphicguy,i'm 2nd post....yee haw!!!!!!!!do i get a badge?
1,June 2014,bwia,"great, but will it be possible to refer to old..."
2,June 2014,imidazol97,"if you have posted in that discussion, click o..."
3,June 2014,stickguy,"if you want to bookmark this one, there is a l..."
4,June 2014,bwia,"thanks @stickguy, it worked like a charm."
5,June 2014,bwia,thanks imidazol97. i will try that later but s...
6,June 2014,Kirstie_H,"no, but it's almost like you're famous. just t..."
7,June 2014,cdnpinhead,"okay, here am i also. now i need to add this t..."
8,June 2014,graphicguy,wow! i'm almost famous....... thanks......
9,June 2014,ab348,"wow, that's a throwback to the old, old, old b..."
